In [15]:
import math

In [16]:
def gini_index(groups,classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score  = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1 - score) * (size / n_instances)
    return gini        

In [17]:
def log_entropy(groups,classes):
    n_instances = float(sum([len(group) for group in groups]))
    total_groups = list()
    ig = 0.0
    for group in groups:
        for row in group:
            total_groups.append(row)
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            if p == 0.0:
                continue
            logp = p * math.log(p,2)
            #print(logp)
            score += (-1)*logp
        ig += (size/n_instances) * score
    #print('ig::',ig)
    score = 0.0
    for class_val in classes:
        p = [row[-1] for row in total_groups].count(class_val) / n_instances
        # print(p)
        logp = p * math.log(p,2)
        #print(logp)
        score += (-1)*logp
    #print('parent entropy',score)
    return score - ig

In [18]:
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0],[1, 0], [1, 0],[1, 1], [1, 1],[1, 1], [1, 1],[1, 1], [1, 1],[1, 1],
                   [1, 1],[1, 1], [1, 1],[1, 1], [1, 1],[1,1]], [[1, 1], [1, 0],[1, 0], [1, 0],[1, 0], [1, 0],[1, 0], 
                                                                 [1, 0],[1, 0], [1, 0],[1, 0], [1, 0],[1, 0]
                                                                ]], [0, 1]))

0.5
0.26546003016591246


In [19]:
#print(log_entropy([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(log_entropy([[[1, 0], [1, 0],[1, 0], [1, 0],[1, 1], [1, 1],[1, 1], [1, 1],[1, 1], [1, 1],[1, 1],
                   [1, 1],[1, 1], [1, 1],[1, 1], [1, 1],[1,1]], [[1, 1], [1, 0],[1, 0], [1, 0],[1, 0], [1, 0],[1, 0], 
                                                                 [1, 0],[1, 0], [1, 0],[1, 0], [1, 0],[1, 0]
                                                                ]], [0, 1]))

0.38121435556157335


In [20]:
def test_split(index,value,dataset):
    left, right = list(),list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left,right

In [50]:
def get_split(dataset,n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index,b_value,b_score,b_groups = 999,999,999,None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0]) - 1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index,row[index],dataset)
            gini = gini_index(groups,class_values)
            #gini = log_entropy(groups,class_values)
            if gini < b_score:
                b_index,b_value,b_score,b_groups = index,row[index],gini,groups
    return {'index' : b_index,'value' : b_value,'groups' : b_groups}

In [22]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes),key = outcomes.count)

In [23]:
def split(node,max_depth,min_size,n_features,depth):
    left,right = node['groups']
    del(node['groups'])
    
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'],node['right'] = to_terminal(left),to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left,n_features)
        split(node['left'],max_depth,min_size,n_features,depth + 1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right,n_features)
        split(node['right'],max_depth,min_size,n_features,depth + 1)

In [24]:
def build_tree(train,max_depth,min_size,n_features):
    root = get_split(train,n_features)
    split(root,max_depth,min_size,n_features,1)
    return root

In [89]:
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))
 
dataset = [[2.771244718,1.784783929,0],
    [1.728571309,1.169761413,0],
    [3.678319846,2.81281357,0],
    [3.961043357,2.61995032,0],
    [2.999208922,2.209014212,0],
    [7.497545867,3.162953546,1],
    [9.00220326,3.339047188,1],
    [7.444542326,0.476683375,1],
    [10.12493903,3.234550982,1],
    [6.642287351,3.319983761,1]]
tree = build_tree(dataset, 3, 1)
print_tree(tree)

0.0
-0.5199666673076944
-0.4711093925305278
ig:: 0.8919684538544
-0.5
-0.5
parent entropy 1.0
-0.5
-0.5
ig:: 1.0
-0.5
-0.5
parent entropy 1.0
0.0
-0.5163871205878868
-0.3467334479787441
ig:: 0.6041843979966417
-0.5
-0.5
parent entropy 1.0
0.0
-0.430827083453526
-0.21919533819482817
ig:: 0.39001345298901247
-0.5
-0.5
parent entropy 1.0
0.0
-0.5306390622295664
-0.4237949406953986
ig:: 0.763547202339972
-0.5
-0.5
parent entropy 1.0
-0.3467334479787441
-0.5163871205878868
0.0
ig:: 0.6041843979966417
-0.5
-0.5
parent entropy 1.0
-0.4237949406953986
-0.5306390622295664
0.0
ig:: 0.763547202339972
-0.5
-0.5
parent entropy 1.0
-0.21919533819482817
-0.430827083453526
0.0
ig:: 0.39001345298901247
-0.5
-0.5
parent entropy 1.0
-0.4711093925305278
-0.5199666673076944
0.0
ig:: 0.8919684538544
-0.5
-0.5
parent entropy 1.0
0.0
0.0
ig:: 0.0
-0.5
-0.5
parent entropy 1.0
-0.5
-0.5
-0.5
-0.5
ig:: 1.0
-0.5
-0.5
parent entropy 1.0
0.0
-0.4711093925305278
-0.5199666673076944
ig:: 0.8919684538544
-0.5
-0.5
par

In [25]:
def predict(node,row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'],dict):
            return predict(node['left'],row)
        else:
            return node['left']
    else:
        if isinstance(node['right'],dict):
            return predict(node['right'],row)
        else:
            return node['right']

In [26]:
def sub_sample(dataset,ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [52]:
def bagging_predict(trees,row):
    predictions = [predict(tree,row) for tree in trees]
    return max(set(predictions),key=predictions.count)

In [28]:
from random import seed
from random import randrange
from csv import reader

In [29]:
def load_csv(filename):
    file = open(filename)
    lines = reader(file)
    dataset = list(lines)
    return dataset

In [30]:
def str_column_to_float(dataset,column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [31]:
def str_column_to_int(dataset,column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i,value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [32]:
def cross_validation_split(dataset,n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i  in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split        

In [33]:
def accuracy_metric(actual,predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.00

In [34]:
def evaulate_algoritham(dataset,algoritham,n_folds,*args):
    folds = cross_validation_split(dataset,n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set,[])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algoritham(train_set,test_set,*args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual,predicted)
        scores.append(accuracy)
    return scores

In [48]:
def random_forest(train,test,max_depth,min_size,sample_size,n_trees,n_features):
    trees = list()
    for i in range(int(n_trees)):
        sample = sub_sample(train,sample_size)
        tree = build_tree(sample,max_depth,min_size,n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees,row) for row in test]
    return predictions

In [ ]:
seed(1)
filename = 'C:\\Users\\rakesh\\Desktop\\ML\\decisiontrees\\sonar.all-data.csv'
dataset = load_csv(filename)
for i in range(0,len(dataset[0]) - 1):
    str_column_to_float(dataset,i)
str_column_to_int(dataset,len(dataset[0]) - 1)
n_folds = 5
max_depth = 5
min_size = 1
sample_size = 1.0
n_features = int(math.sqrt(len(dataset[0]) -1))
for n_trees in [50]:
    scores = evaulate_algoritham(dataset,random_forest,n_folds,max_depth,min_size,n_trees,sample_size,n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))